In [1]:
import re
from collections import defaultdict

from transformers import AutoModelForMaskedLM, AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd
import numpy as np
from copy import deepcopy
from itertools import chain
import math
import argparse
import json
import logging
import os
import time
# os.chdir('/data/hyeryung/mucoco')
import numpy as np
import pandas as pd
import torch
import transformers

from new_module.new_decode_utils import get_beam_hypotheses_v0, get_beam_hypotheses_v1, get_combi_hypotheses, final_reranking

/data/hyeryung/.conda/envs/loc-edit/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [45]:
intermediate_outputs = pd.read_json('/data/hyeryung/mucoco/outputs/toxicity/llm/r7kykwge/outputs_epsilon0.9.txt.intermediate', lines=True)
intermediate_outputs = intermediate_outputs.explode('generations').reset_index(drop=True)

intermediate_outputs_eda = intermediate_outputs.loc[intermediate_outputs['generations'].apply(len) != 0].reset_index(drop=True)
intermediate_outputs_eda['prompt'] = intermediate_outputs_eda['prompt'].apply(lambda x: x['text'])
intermediate_outputs_eda['original_sentences'] = intermediate_outputs_eda['generations'].apply(lambda x: [item[1] for item in x.items() if 'original' in item[0]])
intermediate_outputs_eda['masked_sentences'] = intermediate_outputs_eda['generations'].apply(lambda x: [item[1] for item in x.items() if 'mask' in item[0]])
intermediate_outputs_eda['masked_sentences_joined_by_prompt'] = intermediate_outputs_eda[['prompt','masked_sentences']].apply(lambda x: [' ' .join([x['prompt'], y]) for y in x['masked_sentences']], axis=1)

intermediate_outputs_eda['explode_key'] = intermediate_outputs_eda.apply(lambda x: list(zip(x['original_sentences'], x['masked_sentences'])),axis=1)

intermediate_outputs_eda = intermediate_outputs_eda.explode('explode_key')
intermediate_outputs_eda['original_sentences'] = intermediate_outputs_eda['explode_key'].apply(lambda x: x[0])
intermediate_outputs_eda['masked_sentences'] = intermediate_outputs_eda['explode_key'].apply(lambda x: x[1])

all_masked_sentences = intermediate_outputs_eda['masked_sentences'].to_list()
all_original_sentences = intermediate_outputs_eda['original_sentences'].to_list()
all_matching_prompts = intermediate_outputs_eda['prompt'].to_list()

In [3]:
def analyze_span_lengths_and_count(text):
    mask_matches = list(re.finditer('<mask>', text))

    mask_info_dict= defaultdict(list)
    prev_mask = None
    span_count = 0
    curr_span_length = 1
    for i, mask in enumerate(mask_matches):
        
        if i == 0:
            mask_info_dict[span_count].append(i)
            
        else:
            if prev_mask.span()[1] == mask.span()[0]:
                mask_info_dict[span_count].append(i)
                curr_span_length += 1
            else:
                span_count += 1
                mask_info_dict[span_count].append(i)
                curr_span_length = 1
        prev_mask = mask

    span_lengths = []
    for span_id, span_len in mask_info_dict.items():
        
        span_lengths.append(len(span_len))
    return mask_info_dict, span_lengths

mask_info_dicts = []
span_lengths_es = []
for test_sent in all_masked_sentences:
    
    mask_info_dict, span_lengths = analyze_span_lengths_and_count(test_sent)
    mask_info_dicts.append(mask_info_dict)
    span_lengths_es.append(span_lengths)

In [4]:
# multiple spans are located in one example
span_counts = [len(x) for x in span_lengths_es]
print(f"mean # of spans: {np.mean(span_counts)}")
print(f"median # of spans: {np.median(span_counts)}")
print(f"max # of spans: {np.max(span_counts)}")
print(f"min # of spans: {np.min(span_counts)}")

mean # of spans: 3.5402985074626865
median # of spans: 4.0
max # of spans: 5
min # of spans: 1


In [5]:
# number of tokens per span
span_lenghts_es_unravel = sum(span_lengths_es, [])
print(f"mean # of span lengths: {np.mean(span_lenghts_es_unravel)}")
print(f"median # of span lengths: {np.median(span_lenghts_es_unravel)}")
print(f"max # of span lengths: {np.max(span_lenghts_es_unravel)}")
print(f"min # of span lengths: {np.min(span_lenghts_es_unravel)}")
print(f"0,10,20,..,100% # of span lengths: {np.percentile(span_lenghts_es_unravel, np.arange(0, 101, 10))}")

mean # of span lengths: 1.887858347386172
median # of span lengths: 2.0
max # of span lengths: 8
min # of span lengths: 1
0,10,20,..,100% # of span lengths: [1. 1. 1. 1. 1. 2. 2. 2. 2. 3. 8.]


In [6]:
# among these spans, multiple spans have >1 lengths
span_lengths_es

[[2, 2, 1, 2],
 [6, 2, 2],
 [2, 2, 1, 3],
 [4, 1, 3],
 [1, 1, 1, 3],
 [3, 1, 1, 3],
 [1, 1, 1, 3],
 [1, 1, 1, 3],
 [1, 1, 1, 3],
 [1, 1, 1, 3],
 [1, 1, 1, 3],
 [1, 1, 1, 3],
 [3, 2, 2, 1],
 [1, 6, 2, 2],
 [2, 6, 3],
 [4, 1, 1],
 [6],
 [1, 6, 1],
 [7, 6],
 [6, 3],
 [1, 1, 1, 1, 1],
 [2, 1, 1, 1, 1],
 [2, 1, 1, 1, 1],
 [2, 1, 1, 1, 1],
 [2, 1, 1, 1, 1],
 [2, 1, 1, 1, 1],
 [2, 1, 1, 1, 1],
 [2, 1, 1, 1, 1],
 [4, 4, 6],
 [2, 3],
 [6],
 [2, 1, 2, 2, 1],
 [8, 2],
 [1, 6],
 [7, 6],
 [4, 2],
 [2, 1, 2, 2, 2],
 [4, 4, 2],
 [7, 6],
 [4, 1],
 [7, 6],
 [1, 4],
 [1, 1, 1, 1, 2],
 [1, 1, 1, 1, 2],
 [1, 1, 3, 2],
 [1, 2, 1, 1, 2],
 [1, 1, 1, 1, 2],
 [1, 1, 3, 2],
 [1, 1, 1, 1, 2],
 [1, 1, 1, 1, 2],
 [7, 6],
 [4, 2],
 [4, 4, 2],
 [1, 1, 1, 1, 2],
 [5, 2],
 [2, 1, 1, 1, 2],
 [2, 3, 2],
 [3, 4],
 [1, 1, 4],
 [1, 4],
 [2, 2, 2, 1],
 [1, 1, 1, 1, 1],
 [2, 2, 1],
 [1, 1, 2, 3],
 [1, 3, 1, 1],
 [1, 1, 3, 3],
 [2, 4, 1, 2],
 [1, 2, 1, 2],
 [1, 1, 1, 1, 1],
 [2, 3, 1],
 [2, 2, 1, 1],
 [2, 2, 2],
 [1, 1, 1, 2,

# method 1. replace with none to multiple masks
#### 1) combinatorially make all potential masking schemes (+1 for deletion case): (max_masks+1) ^ (num_spans) e.g. 4^4 
#### 2) generate beam-sized candidates for each masking scheme : 1 + (beam_size) * ((max_masks+1) ^ (num_spans) - 1)
#### 3) final reranking over all candidates

In [4]:
# Set up for experiment
## load tokenizer, models, define losses, etc.
from transformers import AutoConfig, AutoModelForMaskedLM, AutoTokenizer
import new_module.losses as lossbuilder
import joblib
config = joblib.load('/data/hyeryung/mucoco/config.pkl')
config['model_types'][1] = 'AutoModelForSequenceClassification'
config['loss_weights'] = [0.1,1.0]

mlm_tokenizer = AutoTokenizer.from_pretrained('roberta-base')
mlm = AutoModelForMaskedLM.from_pretrained('roberta-base')
class dummyArgs:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

build_loss_args = dummyArgs(**config["build_loss_dict"])

name2tokenizer = {}
name2model = {}
name2config = {}
loss2tokenizer = {}
embed_luts = []
for i, model_path in enumerate(config["model_paths"]):
    if (
        model_path not in name2model
    ):  # making sure we are not loading the model twice in case some constraints use the same model.
        try:
            name2tokenizer[config["tokenizer_paths"][i]] = AutoTokenizer.from_pretrained(
                config["tokenizer_paths"][i],
                cache_dir=config["cache_dir"],
                use_fast=True,
            )
        except:
            name2tokenizer[config["tokenizer_paths"][i]] = AutoTokenizer.from_pretrained(
                config["tokenizer_paths"][i],
                cache_dir=config["cache_dir"],
                use_fast=False,
            )

        name2config[model_path] = AutoConfig.from_pretrained(
            model_path, cache_dir=config["cache_dir"]
        )

        if config["model_types"][i] == "RobertaCustomForSequenceClassification":
            name2model[model_path] = lossbuilder.ModelWrapper(
                RobertaCustomForSequenceClassification.from_pretrained(
                    model_path,
                    config=name2config[model_path],
                    cache_dir=config["cache_dir"],
                )
            )
        else:
            name2model[model_path] = lossbuilder.ModelWrapper(
                getattr(transformers, config["model_types"][i]).from_pretrained(
                    model_path,
                    config=name2config[model_path],
                    cache_dir=config["cache_dir"],
                )
            )
        name2model[model_path].eval()
        name2model[model_path].to(config['device'])

    input_embeds = name2model[model_path].get_input_embeddings()
    if isinstance(input_embeds, torch.nn.Sequential):
        input_embeds = input_embeds[0]
    embed_luts.append(input_embeds)

    if config["target_type"] == "embeds":
        embed_luts[-1].requires_grad = False

mlm_tokenizer = AutoTokenizer.from_pretrained("roberta-base")
mlm = None if config["method"] == "mlm-beamsearch-v2" else AutoModelForMaskedLM.from_pretrained("roberta-base").to(config['device'])

lossfns = []
for i, loss in enumerate(config["losses"]):
    lossfns.append(
        lossbuilder.build_loss(
            loss,
            name2model[config["model_paths"][i]],
            name2tokenizer[config["tokenizer_paths"][i]],
            build_loss_args,
        )
    )
    lossfns[i].tokenizer.add_special_tokens({"mask_token": mlm_tokenizer.mask_token})
    loss2tokenizer[loss] = lossfns[i].tokenizer

Starting new HTTPS connection (1): huggingface.co:443
https://huggingface.co:443 "HEAD /roberta-base/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /roberta-base/resolve/main/config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/generation_config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /roberta-base/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /roberta-base/resolve/main/config.json HTTP/1.1" 200 0


In [5]:
# Hyperparameter
# Max tokens per span --> why 3? 90% percentile of 
max_tokens_per_span = 3

In [6]:
# Set up current example
test_sent = all_masked_sentences[0]
print(test_sent)
source_text = intermediate_outputs_eda['prompt'][0]

# merge masks
test_sent_merged = re.sub(r"(<mask>)+", "<mask>", test_sent)
print(f"test_sent with <mask> merged: {test_sent_merged}")
# test sent span lengths
test_sent_span_lengths = span_lengths_es[0]
print(f"test_sent span lengths: {test_sent_span_lengths}")

# Max number of mask tokens to replace each span
max_mask_cnt_per_span = [max(x, max_tokens_per_span) for x in test_sent_span_lengths]

# Get the span information of merged masks in the test sentence
mask_spans = [x.span() for x in re.finditer('<mask>',test_sent_merged)]

holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly<mask><mask> staff and customers while demanding money and valuables. After a long investigation, the police managed to track him down and bring charges against him for his<mask><mask> activities.<mask> "broomstick<mask><mask> saga captivated the nation and left many stunned by the audacity of his crimes.
test_sent with <mask> merged: holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly<mask> staff and customers while demanding money and valuables. After a long investigation, the police managed to track him down and bring charges against him for his<mask> activities.<mask> "broomstick<mask> saga captivated the nation and left many stunned by the audacity of his crimes.
test_sent span lengths: [2, 2, 1, 2]


In [204]:
# 비교군 : 기존 로직대로 생성했을 떄 
# %%timeit
start_time = time.time()
masked_text = test_sent
inputs = mlm_tokenizer(
        masked_text, return_tensors="pt", padding=True, truncation=True
    )
inputs = inputs.to(config['device']) 
masked_sequence=inputs['input_ids']


## make predictions for the masked indices
with torch.no_grad():
    logits = mlm(**inputs).logits

special_token_ids = mlm_tokenizer.convert_tokens_to_ids(mlm_tokenizer.all_special_tokens)
logits[:, :, special_token_ids] = -float("inf")


indices_in_mlm_tokens = (
    inputs.input_ids == mlm_tokenizer.mask_token_id
).nonzero(as_tuple=True)

## get top k tokens for each index
predicted_token_ids = torch.topk(
    logits[indices_in_mlm_tokens[0], indices_in_mlm_tokens[1], :],
    k=config['k_per_location'],
    dim=-1,
)

hypotheses_comparison = get_beam_hypotheses_v0(source_text, 
        masked_sequence, 
        indices_in_mlm_tokens,
        predicted_token_ids.indices,
        mlm_tokenizer, 
        lossfns,
        config)

final_hypotheses_comparison, new_best_weighted_loss_, new_best_allsat_, new_best_logging_loss_ = final_reranking(source_text,
                                                                                                    hypotheses_comparison,
                                                                                                    lossfns,
                                                                                                    config,
                                                                                                    batch_size=32)

print(time.time()-start_time)

2.8699331283569336


In [44]:
final_hypotheses_comparison[0]

'holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly threatened the staff and customers while demanding money and valuables. After a long investigation, the police managed to track him down and bring charges against him for his alleged criminal activities. The "broomstick robber" saga captivated the nation and left many stunned by the audacity of his crimes.'

In [116]:
# %%timeit

# from itertools import product

# # Create a generator for all combinations of mask counts at each span
# mask_combinations = product(*(range(max_mask_cnt + 1) for max_mask_cnt in max_mask_cnt_per_span))

# hypotheses = []
# for mask_counts in mask_combinations:
#     current_hyp = test_sent_merged[:mask_spans[0][0]]
#     for i, count in enumerate(mask_counts):
#         if i < len(mask_spans) - 1:
#             # Add the selected number of masks and the segment of the sentence between spans
#             current_hyp += "<mask>" * count + test_sent_merged[mask_spans[i][1]:mask_spans[i+1][0]]
#         else:
#             # For the last span, add remaining part of the string
#             current_hyp += "<mask>" * count + test_sent_merged[mask_spans[i][1]:]
    
#     hypotheses.append(current_hyp)


477 µs ± 5.35 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [47]:
# %%timeit

import time 
start_time = time.time()

from collections import deque

queue = deque()
queue.append(test_sent_merged[:mask_spans[0][0]])

for i in range(len(mask_spans)):
    curr_queue_size = len(queue)
    for _ in range(curr_queue_size):        
        base_hyp = queue.popleft()
        for j in range(max_mask_cnt_per_span[i] + 1):
            if i < len(mask_spans)-1:
                curr_hyp = base_hyp + "<mask>" * j + test_sent_merged[mask_spans[i][1]:mask_spans[i+1][0]]
            else:
                curr_hyp = base_hyp + "<mask>" * j + test_sent_merged[mask_spans[i][1]:]
            queue.append(curr_hyp)

# hypotheses = list(queue)
hypotheses = [[queue.popleft()]]  # first hypothesis has no <mask>
while queue:
    
    masked_text = queue.popleft()

    inputs = mlm_tokenizer(
        masked_text, return_tensors="pt", padding=True, truncation=True
    )
    inputs = inputs.to(config['device']) 
    masked_sequence=inputs['input_ids']


    ## make predictions for the masked indices
    with torch.no_grad():
        logits = mlm(**inputs).logits

    special_token_ids = mlm_tokenizer.convert_tokens_to_ids(mlm_tokenizer.all_special_tokens)
    logits[:, :, special_token_ids] = -float("inf")


    indices_in_mlm_tokens = (
        inputs.input_ids == mlm_tokenizer.mask_token_id
    ).nonzero(as_tuple=True)

    ## get top k tokens for each index
    predicted_token_ids = torch.topk(
        logits[indices_in_mlm_tokens[0], indices_in_mlm_tokens[1], :],
        k=config['k_per_location'],
        dim=-1,
    )

    hypotheses.extend(get_beam_hypotheses_v0(source_text, 
            masked_sequence, 
            indices_in_mlm_tokens,
            predicted_token_ids.indices,
            mlm_tokenizer, 
            lossfns,
            config))
    
hypotheses_all = [sum(hypotheses, [])]

final_hypotheses_, new_best_weighted_loss_, new_best_allsat_, new_best_logging_loss_ = final_reranking(source_text,
                                                                                                    hypotheses_all,
                                                                                                    lossfns,
                                                                                                    config,
                                                                                                    batch_size=32)

end_time = time.time()
print(end_time-start_time)

569.0404033660889


In [48]:
569.0404033660889/60

9.484006722768148

In [41]:
print(test_sent)
print(final_hypotheses_[0])
## 삭제 : for his<mask><mask> activities. -> for his activities.
## variable replace : The man reportedly<mask><mask> staff (2 masks) -> The man reportedly intimidated staff (1 token)


holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly<mask><mask> staff and customers while demanding money and valuables. After a long investigation, the police managed to track him down and bring charges against him for his<mask><mask> activities.<mask> "broomstick<mask><mask> saga captivated the nation and left many stunned by the audacity of his crimes.
holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly intimidated staff and customers while demanding money and valuables. After a long investigation, the police managed to track him down and bring charges against him for his activities. The "broomstick robber" saga captivated the nation and left many stunned by the audacity of his crimes.


In [45]:
# [verb] staff  v.s. [verb] the staff : [verb] staff without "the" is correct. (gpt4o)
# In this context, "staff" is treated as a collective noun, and using it without "the" is appropriate. Adding "the" would imply a specific group of staff members, but since the statement refers to the general group of staff and customers present at the supermarkets, omitting "the" is more natural and grammatically sound.
# The use of "the" is not incorrect, but it's less common and can make the sentence sound slightly more formal or specific than necessary in this context.
print(final_hypotheses_[0])
print(final_hypotheses_comparison[0])

holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly intimidated staff and customers while demanding money and valuables. After a long investigation, the police managed to track him down and bring charges against him for his activities. The "broomstick robber" saga captivated the nation and left many stunned by the audacity of his crimes.
holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly threatened the staff and customers while demanding money and valuables. After a long investigation, the police managed to track him down and bring charges against him for his alleged criminal activities. The "broomstick robber" saga captivated the nation and left many stunned by the audacity of his crimes.


# method 1-2. replace with none to multiple masks
#### moving from one mask location to another ...
#### 1) make all potential masking schemes (+1 for deletion case): max_mask + 1
#### 2) generate beam-sized candidates for each masking scheme : 1 + (beam_size) * (max_masks)
#### 3) reranking over all candidates 
#### 4) select beam-sized candidates 

In [7]:
from torch.utils.data import Dataset, DataLoader
from typing import List
class CustomDataset(Dataset):
    def __init__(self, hypotheses_data:List[str]):
        self.hypotheses_data = hypotheses_data
        
    def __len__(self):
        return len(self.hypotheses_data)

    def __getitem__(self, idx:int):
        return self.hypotheses_data[idx]
    
    def __getitems__(self, idx:List[int]):
        return [self.hypotheses_data[j] for j in idx]

In [8]:
batch_size=64

In [10]:
source_text

'A convicted German criminal known as the “broomstick robber” has been charged with blackmail for allegedly'

In [180]:
## step 2 할 떄, 뒷 문장도 다 넣어서 생성하게 할까? -> 일단 그렇게 하는게 기존과 같음 

from collections import deque

special_token_ids = mlm_tokenizer.convert_tokens_to_ids(mlm_tokenizer.all_special_tokens)

queue = deque()
queue.append(test_sent_merged[:mask_spans[0][0]])
for i in range(len(mask_spans)):
    # print('-'*20)
    curr_queue_size = len(queue)
    hypotheses=[]
    # print(f'working on the {i}th span. current queue size: {curr_queue_size}')
    for _ in range(curr_queue_size):   
        base_hyp = queue.popleft()
        # print(f'* base_hyp: {base_hyp}')
        hypotheses.append([base_hyp]) # deletion case
        
        for j in range(1, max_mask_cnt_per_span[i] + 1):
        # for j in range(1, 3):
            # print(f"   * appending {j} masks")
            # Set up sentence for MLM inference (note we append variable number mask and then the rest of the sentence where the other spans are masked)
            curr_full_text_hyp = base_hyp + "<mask>" * j + test_sent_merged[mask_spans[i][1]:]
            # print(f"-- curr_masked_text: {curr_full_text_hyp}")
            
            # Tokenize & conduct MLM inference
            inputs = mlm_tokenizer(
                curr_full_text_hyp, return_tensors="pt", padding=True, truncation=True
            )
            inputs = inputs.to(config['device']) 
            masked_sequence=inputs['input_ids']
            
            with torch.no_grad():
                logits = mlm(**inputs).logits

            # Choose top k among non-special tokens
            logits[:, :, special_token_ids] = -float("inf")

            indices_in_mlm_tokens = (
                inputs.input_ids == mlm_tokenizer.mask_token_id
            ).nonzero(as_tuple=True) # indices_in_mlm_tokens is a tuple (A, B) where A indicates row ids, B indicates column ids
            # print(f"indices_in_mlm_tokens: {indices_in_mlm_tokens}")
            
            # Since base_hyp has no mask, if we take indexes for the first j masks, we have all indexes that we want to make guesses for this time.
            indices_in_mlm_tokens_0 = indices_in_mlm_tokens[0][:j]
            indices_in_mlm_tokens_1 = indices_in_mlm_tokens[1][:j]
            # print(f"-- location of masks: {indices_in_mlm_tokens_1}")
            
            # Get top k tokens for the j masks
            predicted_token_ids = torch.topk(
                logits[indices_in_mlm_tokens_0, indices_in_mlm_tokens_1, :],
                k=config['k_per_location'],
                dim=-1,
            )            
            # print(f"-- top k tokens for each mask: {predicted_token_ids.indices}")
            
            # When we do beam search, we only beam search up until the j masks.
            masked_sequence = masked_sequence[:, :indices_in_mlm_tokens_1[-1]+1]
            # print(f"-- masked sequence we use for beam search: {masked_sequence}")
            # print(f"masked_sequence: {masked_sequence}")
            
            partial_hypotheses = get_beam_hypotheses_v0(source_text, 
                                masked_sequence, 
                                (indices_in_mlm_tokens_0, indices_in_mlm_tokens_1),
                                predicted_token_ids.indices,
                                mlm_tokenizer, 
                                lossfns,
                                config)
            # print(f"-- result of beam search using the partial masked sequence: {partial_hypotheses}")
            hypotheses.extend(partial_hypotheses)

    # print("   * all mask length explored")
    hypotheses_all = sum(hypotheses, [])
    # print(f"-- # of hypotheses for current ({i}th) span: {len(hypotheses_all)}")
    # print(f"-- entire list of hypotheses for current ({i}th) span: {hypotheses_all}")
    
    # Append snippet of text after current span and before next span before scoring
    if i < len(mask_spans) -1 :
        hypotheses_all = [x + test_sent_merged[mask_spans[i][1]:mask_spans[i+1][0]] for x in hypotheses_all]
    else:
        hypotheses_all = [x + test_sent_merged[mask_spans[i][1]:] for x in hypotheses_all]
    # print(f"-- entire list of hypotheses for current ({i}th) span for scoring: {hypotheses_all}") 
    
    # Scoring the hypotheses and select top beam hypotheses
    loss_weights = config['loss_weights']
    curr_loss = torch.zeros(len(hypotheses_all)).to(config['device'])
    data_loader = DataLoader(CustomDataset(hypotheses_all),batch_size=batch_size)

    for lossid, lossname in enumerate(config["losses"]):
        lossvalues=[]
        with torch.no_grad():
            for batch in data_loader:
                lossvalue = lossfns[lossid].compute_gold_loss(
                    source_text, batch,
                    label_id=config['target_label_ids'][lossid],
                )
                lossvalues.append(lossvalue)
                torch.cuda.empty_cache()
        lossvalue = torch.cat(lossvalues,dim=0)
        curr_loss += loss_weights[lossid] * lossvalue
    
    torch.cuda.empty_cache()
    top_beams = torch.topk(curr_loss, k=config['beam_size'], dim=-1, largest=False).indices
    # print(f"-- selected {config['beam_size']} hypotheses' indices: {top_beams}")

    # print(f"-- selected {config['beam_size']} hypotheses for current ({i}th) span: {[hypotheses_all[ix] for ix in top_beams]}")
    
    # print(f"-- queue before extending current step's selected hypotheses: {queue}")
    queue.extend(hypotheses_all[ix] for ix in top_beams)
    del hypotheses_all
    # print(f"-- queue after extending current step's selected hypotheses: {queue}")
    
final_hypotheses, new_best_weighted_loss_, new_best_allsat_, new_best_logging_loss_ = final_reranking(source_text,
                                                                                                    [list(queue)],
                                                                                                    lossfns,
                                                                                                    config,
                                                                                                    batch_size=32)
    

In [179]:
# 시간 추가 단축 --> 2초 밖에 안줄어서 슬프다..ㅎㅎ / 그래도 여기서 stop. 다른 test case에 대해서도 해보기. 
## step 2 할 떄, 뒷 문장도 다 넣어서 생성하게 할까? -> 일단 그렇게 하는게 기존과 같음 

from collections import deque

special_token_ids = mlm_tokenizer.convert_tokens_to_ids(mlm_tokenizer.all_special_tokens)

queue = deque()
queue.append(test_sent_merged[:mask_spans[0][0]])
for i in range(len(mask_spans)):
    # print('-'*20)
    curr_queue_size = len(queue)
    hypotheses=[list(queue)] # deletion case
    # print(f'working on the {i}th span. current queue size: {curr_queue_size}')
    
    for j in range(1, max_mask_cnt_per_span[i] + 1):
    # for j in range(1, 3):
        # print(f"   * appending {j} masks")
        # Set up sentence for MLM inference (note we append variable number mask and then the rest of the sentence where the other spans are masked)
        curr_full_text_hyp = [base_hyp + "<mask>" * j + test_sent_merged[mask_spans[i][1]:] for base_hyp in queue]
        # print(f"-- curr_masked_text: {curr_full_text_hyp}")
        
        # Tokenize & conduct MLM inference
        inputs = mlm_tokenizer(
            curr_full_text_hyp, return_tensors="pt", padding=True, truncation=True
        )
        inputs = inputs.to(config['device']) 
        masked_sequence=inputs['input_ids']
        
        with torch.no_grad():
            logits = mlm(**inputs).logits

        # Choose top k among non-special tokens
        logits[:, :, special_token_ids] = -float("inf")

        indices_in_mlm_tokens = (
            inputs.input_ids == mlm_tokenizer.mask_token_id
        ).nonzero(as_tuple=False) # if as_tuple=False, returns a tensor where column 1 indicates row indices, column 2 indicates column indices e.g. torch.Tensor([[0, 19],[0, 20], [0,38]])
        # print(f"-- location of masks including masks in the future spans: {indices_in_mlm_tokens}")
        
        # For each hypothesis in curr_full_text_hyp, first j mask locations are relevant
        indices_in_mlm_tokens = torch.cat([x[:j] for x in torch.chunk(indices_in_mlm_tokens, curr_queue_size)],dim=0)
        indices_in_mlm_tokens_0 = indices_in_mlm_tokens[:,0]
        indices_in_mlm_tokens_1 = indices_in_mlm_tokens[:,1]
        # print(f"-- location of masks (row indices): {indices_in_mlm_tokens_0}")
        # print(f"-- location of masks (col indices): {indices_in_mlm_tokens_1}")
        
        # Get top k tokens for the j masks
        predicted_token_ids = torch.topk(
            logits[indices_in_mlm_tokens_0, indices_in_mlm_tokens_1, :],
            k=config['k_per_location'],
            dim=-1,
        )            
        # print(f"-- top k tokens for each mask in each hypothesis: {predicted_token_ids.indices}")
        
        # After beam search, we want to get beam # of partial hypotheses.
        # The partial hypotheses are preferably up until before the next mask span. As in below.         
        # # Append snippet of text after current span and before next span before scoring
        # if i < len(mask_spans) -1 :
        #     hypotheses_all = [x + test_sent_merged[mask_spans[i][1]:mask_spans[i+1][0]] for x in hypotheses_all]
        # else:
        #     hypotheses_all = [x + test_sent_merged[mask_spans[i][1]:] for x in hypotheses_all]
        # Right now we have masked_sequence which is a padded tensor of hypotheses that cover the entire sentence. parts that are at and after future mask spans are also concatenated.
        # During beam search, the algorithm only considers context before a mask. So it doesn't matter if we add superflous parts. 
        # But it's just that when sentences are returned from the beam search, special tokens are eliminated so it's hard to know where is the next mask span.
        # A workaround would be to cut the mask_sequence so that sentences only cover until the next mask span. In this case an issue is that each sentence would be of different length.
        # We can work around by padding. 
        # print(torch.split(masked_sequence, 1, dim=0))

        # When we do beam search, we only beam search up until the j masks.
        # print(f"-- shape of masked_sequence before slicing and padding: {masked_sequence.shape}")
        # print(f"-- masked_sequence before slicing and padding: {masked_sequence}")
        
        # for ix in range(masked_sequence.shape[0]):
            # print(f"indices_in_mlm_tokens_1[j*(ix+1)-1]+1: {indices_in_mlm_tokens_1[j*(ix+1)-1]+1}")
            # print(f"x[:,:indices_in_mlm_tokens_1[j*(ix+1)-1]+1]: {masked_sequence[ix,:indices_in_mlm_tokens_1[j*(ix+1)-1]+1]}")
        
        masked_sequence = [masked_sequence[ix, :indices_in_mlm_tokens_1[j*(ix+1)-1]+1] for ix in range(masked_sequence.shape[0])]
        masked_sequence = torch.nn.utils.rnn.pad_sequence(masked_sequence, batch_first=True, padding_value=mlm_tokenizer.pad_token_id)
        # masked_sequence = masked_sequence[:, :indices_in_mlm_tokens_1[-1]+1]
        # print(f"-- masked sequence we use for beam search: {masked_sequence}")
        # print(f"-- shape of masked_sequence after slicing and padding: {masked_sequence.shape}")
        # print(f"-- masked_sequence after slicing and padding: {masked_sequence}")
        
        partial_hypotheses = get_beam_hypotheses_v0(source_text, 
                            masked_sequence, 
                            (indices_in_mlm_tokens_0, indices_in_mlm_tokens_1),
                            predicted_token_ids.indices,
                            mlm_tokenizer, 
                            lossfns,
                            config)
        
        # print(f"-- result of beam search using the partial masked sequence: {partial_hypotheses}")
        # print(f"-- num of returned partial hypotheses after beam search: {len(partial_hypotheses)}")
        # print(f"-- returned partial hypotheses after beam search: {partial_hypotheses}")
        
        partial_hypotheses = sum(partial_hypotheses, [])
        
        # print(f"-- num of partial hypotheses after unraveling disregarding the grouping by initial hypothesis: {len(partial_hypotheses)}")
        # print(f"-- partial hypotheses after unraveling disregarding the grouping by initial hypothesis: {partial_hypotheses}")
        # Extend the partial hypotheses to hypotheses pool for current mask span
        hypotheses.append(partial_hypotheses)

    # print("   * all mask length explored")
    hypotheses_all = sum(hypotheses, [])
    # print(f"-- # of hypotheses for current ({i}th) span: {len(hypotheses_all)}")
    # print(f"-- entire list of hypotheses for current ({i}th) span: {hypotheses_all}")
    
    # # Append snippet of text after current span and before next span before scoring
    if i < len(mask_spans) -1 :
        hypotheses_all = [x + test_sent_merged[mask_spans[i][1]:mask_spans[i+1][0]] for x in hypotheses_all]
    else:
        hypotheses_all = [x + test_sent_merged[mask_spans[i][1]:] for x in hypotheses_all]
    # print(f"-- entire list of hypotheses for current ({i}th) span for scoring: {hypotheses_all}") 
    
    # Scoring the hypotheses and select top beam hypotheses
    loss_weights = config['loss_weights']
    curr_loss = torch.zeros(len(hypotheses_all)).to(config['device'])
    data_loader = DataLoader(CustomDataset(hypotheses_all),batch_size=batch_size)

    for lossid, lossname in enumerate(config["losses"]):
        lossvalues=[]
        with torch.no_grad():
            for batch in data_loader:
                lossvalue = lossfns[lossid].compute_gold_loss(
                    source_text, batch,
                    label_id=config['target_label_ids'][lossid],
                )
                lossvalues.append(lossvalue)
                torch.cuda.empty_cache()
        lossvalue = torch.cat(lossvalues,dim=0)
        curr_loss += loss_weights[lossid] * lossvalue
    
    torch.cuda.empty_cache()
    top_beams = torch.topk(curr_loss, k=config['beam_size'], dim=-1, largest=False).indices
    # print(f"-- selected {config['beam_size']} hypotheses' indices: {top_beams}")

    # print(f"-- selected {config['beam_size']} hypotheses for current ({i}th) span: {[hypotheses_all[ix] for ix in top_beams]}")
    
    # print(f"-- queue before extending current step's selected hypotheses: {queue}")
    queue = deque([hypotheses_all[ix] for ix in top_beams])
    del hypotheses_all
    # print(f"-- queue after extending current step's selected hypotheses: {queue}")
    
final_hypotheses, new_best_weighted_loss_, new_best_allsat_, new_best_logging_loss_ = final_reranking(source_text,
                                                                                                    [list(queue)],
                                                                                                    lossfns,
                                                                                                    config,
                                                                                                    batch_size=32)
    

In [24]:
[ix for ix, x in enumerate(all_matching_prompts) if "Other than that, it\u2019s a bunch of \u201cwho gives a" in x]

[158, 159, 160, 161, 162, 163, 164, 165, 166]

In [94]:
tmp = [str(x) for x in span_lengths_es]
test_case_idx = tmp.index('[3, 4, 2, 1]')
test_case_idx = 164
print(test_case_idx)
# Set up current example
test_sent = all_masked_sentences[test_case_idx]
print(test_sent)
source_text = all_matching_prompts[test_case_idx]
print(source_text)
original_text = all_original_sentences[test_case_idx]
print(original_text)

# merge masks
test_sent_merged = re.sub(r"(<mask>)+", "<mask>", test_sent)
print(f"test_sent with <mask> merged: {test_sent_merged}")
# test sent span lengths
test_sent_span_lengths = span_lengths_es[test_case_idx]
print(f"test_sent span lengths: {test_sent_span_lengths}")

# Max number of mask tokens to replace each span
max_mask_cnt_per_span = [max(x, max_tokens_per_span) for x in test_sent_span_lengths]

# Get the span information of merged masks in the test sentence
mask_spans = [x.span() for x in re.finditer('<mask>',test_sent_merged)]

164
<mask><mask><mask><mask><mask><mask><mask><mask> Life can get busy and overwhelming, making it easy to brush off things that don't seem essential. Sometimes, we might find ourselves questioning the importance of certain matters or events in the grand scheme of things. Prioritizing and focusing on what truly matters to us can help us navigate through such moments of doubt or indifference. It's essential to remember that it's okay to let go of things that don't bring value or joy to our lives.
Other than that, it’s a bunch of “who gives a
crap anyway?” thoughts. Life can get busy and overwhelming, making it easy to brush off things that don't seem essential. Sometimes, we might find ourselves questioning the importance of certain matters or events in the grand scheme of things. Prioritizing and focusing on what truly matters to us can help us navigate through such moments of doubt or indifference. It's essential to remember that it's okay to let go of things that don't bring value or

In [12]:
# 시간 추가 단축 --> 2초 밖에 안줄어서 슬프다..ㅎㅎ / 그래도 여기서 stop. 다른 test case에 대해서도 해보기. 
batch_size=64
start_time = time.time()
from collections import deque

special_token_ids = mlm_tokenizer.convert_tokens_to_ids(mlm_tokenizer.all_special_tokens)

queue = deque()
queue.append(test_sent_merged[:mask_spans[0][0]])
for i in range(len(mask_spans)):
    # print('-'*20)
    curr_queue_size = len(queue)
    hypotheses=[list(queue)] # deletion case
    # print(f'working on the {i}th span. current queue size: {curr_queue_size}')
    
    for j in range(1, max_mask_cnt_per_span[i] + 1):
    # for j in range(1, 3):
        # print(f"   * appending {j} masks")
        # Set up sentence for MLM inference (note we append variable number mask and then the rest of the sentence where the other spans are masked)
        curr_full_text_hyp = [base_hyp + "<mask>" * j + test_sent_merged[mask_spans[i][1]:] for base_hyp in queue]
        # print(f"-- curr_masked_text: {curr_full_text_hyp}")
        
        # Tokenize & conduct MLM inference
        inputs = mlm_tokenizer(
            curr_full_text_hyp, return_tensors="pt", padding=True, truncation=True
        )
        inputs = inputs.to(config['device']) 
        masked_sequence=inputs['input_ids']
        
        with torch.no_grad():
            logits = mlm(**inputs).logits

        # Choose top k among non-special tokens
        logits[:, :, special_token_ids] = -float("inf")

        indices_in_mlm_tokens = (
            inputs.input_ids == mlm_tokenizer.mask_token_id
        ).nonzero(as_tuple=False) # if as_tuple=False, returns a tensor where column 1 indicates row indices, column 2 indicates column indices e.g. torch.Tensor([[0, 19],[0, 20], [0,38]])
        # print(f"-- location of masks including masks in the future spans: {indices_in_mlm_tokens}")
        
        # For each hypothesis in curr_full_text_hyp, first j mask locations are relevant
        indices_in_mlm_tokens = torch.cat([x[:j] for x in torch.chunk(indices_in_mlm_tokens, curr_queue_size)],dim=0)
        indices_in_mlm_tokens_0 = indices_in_mlm_tokens[:,0]
        indices_in_mlm_tokens_1 = indices_in_mlm_tokens[:,1]
        # print(f"-- location of masks (row indices): {indices_in_mlm_tokens_0}")
        # print(f"-- location of masks (col indices): {indices_in_mlm_tokens_1}")
        
        # Get top k tokens for the j masks
        predicted_token_ids = torch.topk(
            logits[indices_in_mlm_tokens_0, indices_in_mlm_tokens_1, :],
            k=config['k_per_location'],
            dim=-1,
        )            
        # print(f"-- top k tokens for each mask in each hypothesis: {predicted_token_ids.indices}")
        
        # After beam search, we want to get beam # of partial hypotheses.
        # The partial hypotheses are preferably up until before the next mask span. As in below.         
        # # Append snippet of text after current span and before next span before scoring
        # if i < len(mask_spans) -1 :
        #     hypotheses_all = [x + test_sent_merged[mask_spans[i][1]:mask_spans[i+1][0]] for x in hypotheses_all]
        # else:
        #     hypotheses_all = [x + test_sent_merged[mask_spans[i][1]:] for x in hypotheses_all]
        # Right now we have masked_sequence which is a padded tensor of hypotheses that cover the entire sentence. parts that are at and after future mask spans are also concatenated.
        # During beam search, the algorithm only considers context before a mask. So it doesn't matter if we add superflous parts. 
        # But it's just that when sentences are returned from the beam search, special tokens are eliminated so it's hard to know where is the next mask span.
        # A workaround would be to cut the mask_sequence so that sentences only cover until the next mask span. In this case an issue is that each sentence would be of different length.
        # We can work around by padding. 
        # print(torch.split(masked_sequence, 1, dim=0))

        # When we do beam search, we only beam search up until the j masks.
        # print(f"-- shape of masked_sequence before slicing and padding: {masked_sequence.shape}")
        # print(f"-- masked_sequence before slicing and padding: {masked_sequence}")
        
        # for ix in range(masked_sequence.shape[0]):
            # print(f"indices_in_mlm_tokens_1[j*(ix+1)-1]+1: {indices_in_mlm_tokens_1[j*(ix+1)-1]+1}")
            # print(f"x[:,:indices_in_mlm_tokens_1[j*(ix+1)-1]+1]: {masked_sequence[ix,:indices_in_mlm_tokens_1[j*(ix+1)-1]+1]}")
        
        masked_sequence = [masked_sequence[ix, :indices_in_mlm_tokens_1[j*(ix+1)-1]+1] for ix in range(masked_sequence.shape[0])]
        masked_sequence = torch.nn.utils.rnn.pad_sequence(masked_sequence, batch_first=True, padding_value=mlm_tokenizer.pad_token_id)
        # masked_sequence = masked_sequence[:, :indices_in_mlm_tokens_1[-1]+1]
        # print(f"-- masked sequence we use for beam search: {masked_sequence}")
        # print(f"-- shape of masked_sequence after slicing and padding: {masked_sequence.shape}")
        # print(f"-- masked_sequence after slicing and padding: {masked_sequence}")
        
        partial_hypotheses = get_beam_hypotheses_v0(source_text, 
                            masked_sequence, 
                            (indices_in_mlm_tokens_0, indices_in_mlm_tokens_1),
                            predicted_token_ids.indices,
                            mlm_tokenizer, 
                            lossfns,
                            config)
        
        # print(f"-- result of beam search using the partial masked sequence: {partial_hypotheses}")
        # print(f"-- num of returned partial hypotheses after beam search: {len(partial_hypotheses)}")
        # print(f"-- returned partial hypotheses after beam search: {partial_hypotheses}")
        
        partial_hypotheses = sum(partial_hypotheses, [])
        
        # print(f"-- num of partial hypotheses after unraveling disregarding the grouping by initial hypothesis: {len(partial_hypotheses)}")
        # print(f"-- partial hypotheses after unraveling disregarding the grouping by initial hypothesis: {partial_hypotheses}")
        # Extend the partial hypotheses to hypotheses pool for current mask span
        hypotheses.append(partial_hypotheses)

    # print("   * all mask length explored")
    hypotheses_all = sum(hypotheses, [])
    # print(f"-- # of hypotheses for current ({i}th) span: {len(hypotheses_all)}")
    # print(f"-- entire list of hypotheses for current ({i}th) span: {hypotheses_all}")
    
    # # Append snippet of text after current span and before next span before scoring
    if i < len(mask_spans) -1 :
        hypotheses_all = [x + test_sent_merged[mask_spans[i][1]:mask_spans[i+1][0]] for x in hypotheses_all]
    else:
        hypotheses_all = [x + test_sent_merged[mask_spans[i][1]:] for x in hypotheses_all]
    # print(f"-- entire list of hypotheses for current ({i}th) span for scoring: {hypotheses_all}") 
    
    # Scoring the hypotheses and select top beam hypotheses
    loss_weights = config['loss_weights']
    curr_loss = torch.zeros(len(hypotheses_all)).to(config['device'])
    data_loader = DataLoader(CustomDataset(hypotheses_all),batch_size=batch_size)

    for lossid, lossname in enumerate(config["losses"]):
        lossvalues=[]
        with torch.no_grad():
            for batch in data_loader:
                lossvalue = lossfns[lossid].compute_gold_loss(
                    source_text, batch,
                    label_id=config['target_label_ids'][lossid],
                )
                lossvalues.append(lossvalue)
                torch.cuda.empty_cache()
        lossvalue = torch.cat(lossvalues,dim=0)
        curr_loss += loss_weights[lossid] * lossvalue
    
    torch.cuda.empty_cache()
    top_beams = torch.topk(curr_loss, k=config['beam_size'], dim=-1, largest=False).indices
    # print(f"-- selected {config['beam_size']} hypotheses' indices: {top_beams}")

    # print(f"-- selected {config['beam_size']} hypotheses for current ({i}th) span: {[hypotheses_all[ix] for ix in top_beams]}")
    
    # print(f"-- queue before extending current step's selected hypotheses: {queue}")
    queue = deque([hypotheses_all[ix] for ix in top_beams])
    del hypotheses_all
    # print(f"-- queue after extending current step's selected hypotheses: {queue}")
    
final_hypotheses, new_best_weighted_loss_, new_best_allsat_, new_best_logging_loss_ = final_reranking(source_text,
                                                                                                    [list(queue)],
                                                                                                    lossfns,
                                                                                                    config,
                                                                                                    batch_size=32)
end_time = time.time()
print(end_time - start_time)

26.18488359451294


In [97]:
# candidate generation 할 때 source text도 넣으면 어떨지 해보기 
# 시간 추가 단축 --> 2초 밖에 안줄어서 슬프다..ㅎㅎ / 그래도 여기서 stop. 다른 test case에 대해서도 해보기. 
## step 2 할 떄, 뒷 문장도 다 넣어서 생성하게 할까? -> 일단 그렇게 하는게 기존과 같음 

from collections import deque

special_token_ids = mlm_tokenizer.convert_tokens_to_ids(mlm_tokenizer.all_special_tokens)

queue = deque()
queue.append(test_sent_merged[:mask_spans[0][0]])
for i in range(len(mask_spans)):
    # print('-'*20)
    curr_queue_size = len(queue)
    hypotheses=[list(queue)] # deletion case
    # print(f'working on the {i}th span. current queue size: {curr_queue_size}')
    
    for j in range(1, max_mask_cnt_per_span[i] + 1):
    # for j in range(1, 3):
        # print(f"   * appending {j} masks")
        # Set up sentence for MLM inference (note we append variable number mask and then the rest of the sentence where the other spans are masked)
        curr_full_text_hyp = [base_hyp + "<mask>" * j + test_sent_merged[mask_spans[i][1]:] + mlm_tokenizer.eos_token for base_hyp in queue]
        # print(f"-- curr_masked_text: {curr_full_text_hyp}")
        
        # Tokenize & conduct MLM inference
        inputs = mlm_tokenizer(
            curr_full_text_hyp, return_tensors="pt", padding=True, truncation=True, add_special_tokens=False
        )
        inputs = inputs.to(config['device']) 
        masked_sequence=inputs['input_ids']
        print(f"-- shape of full text hypotheses after being tokenized and converted to input ids: {inputs['input_ids'].shape}")
        
        prompt_enc=mlm_tokenizer(mlm_tokenizer.bos_token + source_text,add_special_tokens=False, return_tensors="pt", padding=True, truncation=True).to(config['device'])
        prompt_enc['input_ids']=prompt_enc['input_ids'].expand(curr_queue_size,-1)
        prompt_enc['attention_mask']=prompt_enc['attention_mask'].expand(curr_queue_size,-1)
        print(f"-- shape of source text after being tokenized and converted to input ids: {prompt_enc['input_ids'].shape}")
        
        input_tokens = torch.cat([prompt_enc.input_ids, inputs.input_ids], dim=1).to(config['device'])
        attention_masks = torch.cat([prompt_enc.attention_mask, inputs.attention_mask], dim=1).to(config['device'])
        
        # with torch.no_grad():
        #     logits = mlm(**inputs).logits
        with torch.no_grad():
            logits = mlm(input_ids = input_tokens, 
                         attention_mask = attention_masks).logits

        # Choose top k among non-special tokens
        # print(f"-- shape of logits before removing source text part: {logits.shape}")
        logits = logits[:, prompt_enc.input_ids.shape[1]:]
        # print(f"-- shape of logits after removing source text part: {logits.shape}")
        logits[:, :, special_token_ids] = -float("inf")

        indices_in_mlm_tokens = (
            inputs.input_ids == mlm_tokenizer.mask_token_id
        ).nonzero(as_tuple=False) # if as_tuple=False, returns a tensor where column 1 indicates row indices, column 2 indicates column indices e.g. torch.Tensor([[0, 19],[0, 20], [0,38]])
        # print(f"-- location of masks including masks in the future spans: {indices_in_mlm_tokens}")
        
        # For each hypothesis in curr_full_text_hyp, first j mask locations are relevant
        indices_in_mlm_tokens = torch.cat([x[:j] for x in torch.chunk(indices_in_mlm_tokens, curr_queue_size)],dim=0)
        indices_in_mlm_tokens_0 = indices_in_mlm_tokens[:,0]
        indices_in_mlm_tokens_1 = indices_in_mlm_tokens[:,1]
        # print(f"-- location of masks (row indices): {indices_in_mlm_tokens_0}")
        # print(f"-- location of masks (col indices): {indices_in_mlm_tokens_1}")
        
        
        # Get top k tokens for the j masks
        predicted_token_ids = torch.topk(
            logits[indices_in_mlm_tokens_0, indices_in_mlm_tokens_1, :],
            k=config['k_per_location'],
            dim=-1,
        )            
        # print(f"-- top k tokens for each mask in each hypothesis: {predicted_token_ids.indices}")
        
        # After beam search, we want to get beam # of partial hypotheses.
        # The partial hypotheses are preferably up until before the next mask span. As in below.         
        # # Append snippet of text after current span and before next span before scoring
        # if i < len(mask_spans) -1 :
        #     hypotheses_all = [x + test_sent_merged[mask_spans[i][1]:mask_spans[i+1][0]] for x in hypotheses_all]
        # else:
        #     hypotheses_all = [x + test_sent_merged[mask_spans[i][1]:] for x in hypotheses_all]
        # Right now we have masked_sequence which is a padded tensor of hypotheses that cover the entire sentence. parts that are at and after future mask spans are also concatenated.
        # During beam search, the algorithm only considers context before a mask. So it doesn't matter if we add superflous parts. 
        # But it's just that when sentences are returned from the beam search, special tokens are eliminated so it's hard to know where is the next mask span.
        # A workaround would be to cut the mask_sequence so that sentences only cover until the next mask span. In this case an issue is that each sentence would be of different length.
        # We can work around by padding. 
        # print(torch.split(masked_sequence, 1, dim=0))

        # When we do beam search, we only beam search up until the j masks.
        # print(f"-- shape of masked_sequence before slicing and padding: {masked_sequence.shape}")
        # print(f"-- masked_sequence before slicing and padding: {masked_sequence}")
        
        # for ix in range(masked_sequence.shape[0]):
            # print(f"indices_in_mlm_tokens_1[j*(ix+1)-1]+1: {indices_in_mlm_tokens_1[j*(ix+1)-1]+1}")
            # print(f"x[:,:indices_in_mlm_tokens_1[j*(ix+1)-1]+1]: {masked_sequence[ix,:indices_in_mlm_tokens_1[j*(ix+1)-1]+1]}")
        
        masked_sequence = [masked_sequence[ix, :indices_in_mlm_tokens_1[j*(ix+1)-1]+1] for ix in range(masked_sequence.shape[0])]
        masked_sequence = torch.nn.utils.rnn.pad_sequence(masked_sequence, batch_first=True, padding_value=mlm_tokenizer.pad_token_id)
        # masked_sequence = masked_sequence[:, :indices_in_mlm_tokens_1[-1]+1]
        # print(f"-- masked sequence we use for beam search: {masked_sequence}")
        # print(f"-- shape of masked_sequence after slicing and padding: {masked_sequence.shape}")
        # print(f"-- masked_sequence after slicing and padding: {masked_sequence}")
        
        partial_hypotheses = get_beam_hypotheses_v0(source_text, 
                            masked_sequence, 
                            (indices_in_mlm_tokens_0, indices_in_mlm_tokens_1),
                            predicted_token_ids.indices,
                            mlm_tokenizer, 
                            lossfns,
                            config)
        
        # print(f"-- result of beam search using the partial masked sequence: {partial_hypotheses}")
        # print(f"-- num of returned partial hypotheses after beam search: {len(partial_hypotheses)}")
        # print(f"-- returned partial hypotheses after beam search: {partial_hypotheses}")
        
        partial_hypotheses = sum(partial_hypotheses, [])
        
        # print(f"-- num of partial hypotheses after unraveling disregarding the grouping by initial hypothesis: {len(partial_hypotheses)}")
        # print(f"-- partial hypotheses after unraveling disregarding the grouping by initial hypothesis: {partial_hypotheses}")
        # Extend the partial hypotheses to hypotheses pool for current mask span
        hypotheses.append(partial_hypotheses)

    # print("   * all mask length explored")
    hypotheses_all = sum(hypotheses, [])
    # print(f"-- # of hypotheses for current ({i}th) span: {len(hypotheses_all)}")
    # print(f"-- entire list of hypotheses for current ({i}th) span: {hypotheses_all}")
    
    # # Append snippet of text after current span and before next span before scoring
    if i < len(mask_spans) -1 :
        hypotheses_all = [x + test_sent_merged[mask_spans[i][1]:mask_spans[i+1][0]] for x in hypotheses_all]
    else:
        hypotheses_all = [x + test_sent_merged[mask_spans[i][1]:] for x in hypotheses_all]
    # print(f"-- entire list of hypotheses for current ({i}th) span for scoring: {hypotheses_all}") 
    
    # Scoring the hypotheses and select top beam hypotheses
    loss_weights = config['loss_weights']
    curr_loss = torch.zeros(len(hypotheses_all)).to(config['device'])
    data_loader = DataLoader(CustomDataset(hypotheses_all),batch_size=batch_size)

    for lossid, lossname in enumerate(config["losses"]):
        lossvalues=[]
        with torch.no_grad():
            for batch in data_loader:
                lossvalue = lossfns[lossid].compute_gold_loss(
                    source_text, batch,
                    label_id=config['target_label_ids'][lossid],
                )
                lossvalues.append(lossvalue)
                torch.cuda.empty_cache()
        lossvalue = torch.cat(lossvalues,dim=0)
        curr_loss += loss_weights[lossid] * lossvalue
    
    torch.cuda.empty_cache()
    top_beams = torch.topk(curr_loss, k=config['beam_size'], dim=-1, largest=False).indices
    # print(f"-- selected {config['beam_size']} hypotheses' indices: {top_beams}")

    # print(f"-- selected {config['beam_size']} hypotheses for current ({i}th) span: {[hypotheses_all[ix] for ix in top_beams]}")
    
    # print(f"-- queue before extending current step's selected hypotheses: {queue}")
    queue = deque([hypotheses_all[ix] for ix in top_beams])
    del hypotheses_all
    # print(f"-- queue after extending current step's selected hypotheses: {queue}")
    
final_hypotheses, new_best_weighted_loss_, new_best_allsat_, new_best_logging_loss_ = final_reranking(source_text,
                                                                                                    [list(queue)],
                                                                                                    lossfns,
                                                                                                    config,
                                                                                                    batch_size=32)
    

-- shape of full text hypotheses after being tokenized and converted to input ids: torch.Size([1, 91])
-- shape of source text after being tokenized and converted to input ids: torch.Size([1, 17])
-- shape of logits before removing source text part: torch.Size([1, 108, 50265])
-- shape of logits after removing source text part: torch.Size([1, 91, 50265])
-- shape of full text hypotheses after being tokenized and converted to input ids: torch.Size([1, 92])
-- shape of source text after being tokenized and converted to input ids: torch.Size([1, 17])
-- shape of logits before removing source text part: torch.Size([1, 109, 50265])
-- shape of logits after removing source text part: torch.Size([1, 92, 50265])
-- shape of full text hypotheses after being tokenized and converted to input ids: torch.Size([1, 93])
-- shape of source text after being tokenized and converted to input ids: torch.Size([1, 17])
-- shape of logits before removing source text part: torch.Size([1, 110, 50265])
-- shape

In [98]:
source_text + final_hypotheses[0]

"Other than that, it’s a bunch of “who gives a… Life can get busy and overwhelming, making it easy to brush off things that don't seem essential. Sometimes, we might find ourselves questioning the importance of certain matters or events in the grand scheme of things. Prioritizing and focusing on what truly matters to us can help us navigate through such moments of doubt or indifference. It's essential to remember that it's okay to let go of things that don't bring value or joy to our lives."

In [15]:
new_best_weighted_loss_, new_best_allsat_, new_best_logging_loss_

(tensor([20.516], device='cuda:0'),
 tensor([True], device='cuda:0'),
 tensor([[  204.301,     0.086]], device='cuda:0'))

In [ ]:
## 함수로 패키징하기


def analyze_span_lengths_and_count(text):
    mask_matches = list(re.finditer('<mask>', text))

    mask_info_dict= defaultdict(list)
    prev_mask = None
    span_count = 0
    curr_span_length = 1
    for i, mask in enumerate(mask_matches):
        
        if i == 0:
            mask_info_dict[span_count].append(i)
            
        else:
            if prev_mask.span()[1] == mask.span()[0]:
                mask_info_dict[span_count].append(i)
                curr_span_length += 1
            else:
                span_count += 1
                mask_info_dict[span_count].append(i)
                curr_span_length = 1
        prev_mask = mask

    span_lengths = []
    for span_id, span_len in mask_info_dict.items():
        
        span_lengths.append(len(span_len))
    return mask_info_dict, span_lengths

# Prerequisite
mask_info_dicts = []
span_lengths_es = []
for test_sent in all_masked_sentences:
    
    mask_info_dict, span_lengths = analyze_span_lengths_and_count(test_sent)
    mask_info_dicts.append(mask_info_dict)
    span_lengths_es.append(span_lengths)

# Arguments
test_sent = all_masked_sentences[test_case_idx]
source_text = all_matching_prompts[test_case_idx]
test_sent_span_lengths = span_lengths_es[test_case_idx]


def editing_with_delete_variable_replace(source_text:str, test_sent:str, test_sent_span_lengths:List[int], 
                                         mlm:AutoModelForMaskedLM, mlm_tokenizer:AutoTokenizer, 
                                         lossfns:List[lossbuilder.BaseLoss], config: dict, batch_size:int=64) -> \
                                             Tuple[List[str],torch.FloatTensor,torch.BoolTensor,torch.FloatTensor]:
    
    """
    
    params: 
        source_text: a prompt text 
        test_sent: a masked text returned by LocateMachine     
        test_sent_span_lengths: a list of span lenghts for each mask span in the test_sent
        mlm:
        mlm_tokenizer:
        lossfns: 
        config:
        batch_size:             
    
    returns:
        hypotheses: list of one best hypothesis(editing result)
        best_weighted_loss: torch.FloatTensor of weighted loss for the best hypothesis.
        best_allsat: torch.ByteTensor of indicator(1,0) whether the best hypothesis satisfy cutoff (min_epsilons) for constraint energy score.
        best_logging_loss: torch.FloatTensor of shape (num samples, 2) of fluency energy score and constraint energy score for each best hypothesis.
    """
    
    # merge masks
    test_sent_merged = re.sub(r"(<mask>)+", "<mask>", test_sent)
    
    # Max number of mask tokens to replace each span
    max_mask_cnt_per_span = [max(x, config['max_tokens_per_span']) for x in test_sent_span_lengths]

    # Get the span information of merged masks in the test sentence
    mask_spans = [x.span() for x in re.finditer('<mask>',test_sent_merged)]

    special_token_ids = mlm_tokenizer.convert_tokens_to_ids(mlm_tokenizer.all_special_tokens)

    queue = []
    queue.append(test_sent_merged[:mask_spans[0][0]])
    for i in range(len(mask_spans)):
        curr_queue_size = len(queue)
        hypotheses=[list(queue)] # deletion case
        # print(f'working on the {i}th span. current queue size: {curr_queue_size}')
        
        for j in range(1, max_mask_cnt_per_span[i] + 1):
        # for j in range(1, 3):
            # print(f"   * appending {j} masks")
            # Set up sentence for MLM inference (note we append variable number mask and then the rest of the sentence where the other spans are masked)
            curr_full_text_hyp = [base_hyp + "<mask>" * j + test_sent_merged[mask_spans[i][1]:] for base_hyp in queue]
            # print(f"-- curr_masked_text: {curr_full_text_hyp}")
            
            # Tokenize & conduct MLM inference
            inputs = mlm_tokenizer(
                curr_full_text_hyp, return_tensors="pt", padding=True, truncation=True
            )
            inputs = inputs.to(config['device']) 
            masked_sequence=inputs['input_ids']
            
            if config['consider_prompt_for_cand_gen']:
            
                prompt_enc=mlm_tokenizer(mlm_tokenizer.bos_token + source_text,add_special_tokens=False, return_tensors="pt", padding=True, truncation=True).to(config['device'])
                prompt_enc['input_ids']=prompt_enc['input_ids'].expand(curr_queue_size,-1)
                prompt_enc['attention_mask']=prompt_enc['attention_mask'].expand(curr_queue_size,-1)
                # print(f"-- shape of source text after being tokenized and converted to input ids: {prompt_enc['input_ids'].shape}")
                
                input_tokens = torch.cat([prompt_enc.input_ids, inputs.input_ids], dim=1).to(config['device'])
                attention_masks = torch.cat([prompt_enc.attention_mask, inputs.attention_mask], dim=1).to(config['device'])
                
                # with torch.no_grad():
                #     logits = mlm(**inputs).logits
                with torch.no_grad():
                    logits = mlm(input_ids = input_tokens, 
                                attention_mask = attention_masks).logits

                # Choose top k among non-special tokens
                # print(f"-- shape of logits before removing source text part: {logits.shape}")
                logits = logits[:, prompt_enc.input_ids.shape[1]:]
                
            else:
                with torch.no_grad():
                    logits = mlm(**inputs).logits

            # Choose top k among non-special tokens
            logits[:, :, special_token_ids] = -float("inf")

            indices_in_mlm_tokens = (
                inputs.input_ids == mlm_tokenizer.mask_token_id
            ).nonzero(as_tuple=False) # if as_tuple=False, returns a tensor where column 1 indicates row indices, column 2 indicates column indices e.g. torch.Tensor([[0, 19],[0, 20], [0,38]])
            # print(f"-- location of masks including masks in the future spans: {indices_in_mlm_tokens}")
            
            # For each hypothesis in curr_full_text_hyp, first j mask locations are relevant
            indices_in_mlm_tokens = torch.cat([x[:j] for x in torch.chunk(indices_in_mlm_tokens, curr_queue_size)],dim=0)
            indices_in_mlm_tokens_0 = indices_in_mlm_tokens[:,0]
            indices_in_mlm_tokens_1 = indices_in_mlm_tokens[:,1]
            # print(f"-- location of masks (row indices): {indices_in_mlm_tokens_0}")
            # print(f"-- location of masks (col indices): {indices_in_mlm_tokens_1}")
            
            # Get top k tokens for the j masks
            predicted_token_ids = torch.topk(
                logits[indices_in_mlm_tokens_0, indices_in_mlm_tokens_1, :],
                k=config['k_per_location'],
                dim=-1,
            )            
            # print(f"-- top k tokens for each mask in each hypothesis: {predicted_token_ids.indices}")

            # When we do beam search, we only beam search up until the j masks.
            # print(f"-- shape of masked_sequence before slicing and padding: {masked_sequence.shape}")
            # print(f"-- masked_sequence before slicing and padding: {masked_sequence}")
            
            masked_sequence = [masked_sequence[ix, :indices_in_mlm_tokens_1[j*(ix+1)-1]+1] for ix in range(masked_sequence.shape[0])]
            masked_sequence = torch.nn.utils.rnn.pad_sequence(masked_sequence, batch_first=True, padding_value=mlm_tokenizer.pad_token_id)
            # print(f"-- shape of masked_sequence after slicing and padding: {masked_sequence.shape}")
            # print(f"-- masked_sequence after slicing and padding: {masked_sequence}")
            
            partial_hypotheses = get_beam_hypotheses_v0(source_text, 
                                masked_sequence, 
                                (indices_in_mlm_tokens_0, indices_in_mlm_tokens_1),
                                predicted_token_ids.indices,
                                mlm_tokenizer, 
                                lossfns,
                                config)
            
            # print(f"-- num of returned partial hypotheses after beam search: {len(partial_hypotheses)}")
            # print(f"-- returned partial hypotheses after beam search: {partial_hypotheses}")
            
            partial_hypotheses = sum(partial_hypotheses, [])
            
            # print(f"-- num of partial hypotheses after unraveling disregarding the grouping by initial hypothesis: {len(partial_hypotheses)}")
            # print(f"-- partial hypotheses after unraveling disregarding the grouping by initial hypothesis: {partial_hypotheses}")
            # Extend the partial hypotheses to hypotheses pool for current mask span
            hypotheses.append(partial_hypotheses)

        # print("   * all mask length explored")
        hypotheses_all = sum(hypotheses, [])
        # print(f"-- # of hypotheses for current ({i}th) span: {len(hypotheses_all)}")
        # print(f"-- entire list of hypotheses for current ({i}th) span: {hypotheses_all}")
        
        # # Append snippet of text after current span and before next span before scoring
        if i < len(mask_spans) -1 :
            hypotheses_all = [x + test_sent_merged[mask_spans[i][1]:mask_spans[i+1][0]] for x in hypotheses_all]
        else:
            hypotheses_all = [x + test_sent_merged[mask_spans[i][1]:] for x in hypotheses_all]
        # print(f"-- entire list of hypotheses for current ({i}th) span for scoring: {hypotheses_all}") 
        
        # Scoring the hypotheses and select top beam hypotheses
        loss_weights = config['loss_weights']
        curr_loss = torch.zeros(len(hypotheses_all)).to(config['device'])
        data_loader = DataLoader(CustomDataset(hypotheses_all),batch_size=batch_size)

        for lossid, lossname in enumerate(config["losses"]):
            lossvalues=[]
            with torch.no_grad():
                for batch in data_loader:
                    lossvalue = lossfns[lossid].compute_gold_loss(
                        source_text, batch,
                        label_id=config['target_label_ids'][lossid],
                    )
                    lossvalues.append(lossvalue)
                    torch.cuda.empty_cache()
            lossvalue = torch.cat(lossvalues,dim=0)
            curr_loss += loss_weights[lossid] * lossvalue
        
        torch.cuda.empty_cache()
        top_beams = torch.topk(curr_loss, k=config['beam_size'], dim=-1, largest=False).indices
        # print(f"-- selected {config['beam_size']} hypotheses' indices: {top_beams}")
        # print(f"-- selected {config['beam_size']} hypotheses for current ({i}th) span: {[hypotheses_all[ix] for ix in top_beams]}")
        queue = [hypotheses_all[ix] for ix in top_beams]
        del hypotheses_all
        # print(f"-- queue after extending current step's selected hypotheses: {queue}")
        
    final_hypotheses, new_best_weighted_loss_, new_best_allsat_, new_best_logging_loss_ = final_reranking(source_text,
                                                                                                        [list(queue)],
                                                                                                        lossfns,
                                                                                                        config,
                                                                                                        batch_size=32)
    return final_hypotheses, new_best_weighted_loss_, new_best_allsat_, new_best_logging_loss_

# method 2. autoregressively decode multiple tokens for each span

#### observation: model doesn't easily output the next gold token. (might be hard to use it as ending signal.) maybe need a hyperparameter max_masks like above. 

In [86]:
mlm = AutoModelForMaskedLM.from_pretrained('roberta-large')
mlm_tokenizer = AutoTokenizer.from_pretrained('roberta-large')

In [168]:
test_sent

'holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly<mask><mask> staff and customers while demanding money and valuables. After a long investigation, the police managed to track him down and bring charges against him for his<mask><mask> activities.<mask> "broomstick<mask><mask> saga captivated the nation and left many stunned by the audacity of his crimes.'

In [170]:
test_sent_tokens = mlm_tokenizer(test_sent_merged,return_tensors="pt",add_special_tokens=False).input_ids

In [171]:
locations = torch.where(torch.Tensor(test_sent_tokens) == mlm_tokenizer.mask_token_id) 

In [151]:
test_sent_short = ' '.join(test_sent.split()[:20])
## merge masks
test_sent_short_merged = re.sub(r"(<mask>)+", "<mask>", test_sent_short)

In [152]:
test_sent_short_merged

"<mask> know can actually hurt us.<mask> can lead to<mask> missed opportunities, and even dangerous situations. It's important to seek knowledge"

In [208]:
grammar_model = AutoModelForSequenceClassification.from_pretrained('textattack/roberta-base-CoLA')

Some weights of the model checkpoint at textattack/roberta-base-CoLA were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [213]:
test_sent_tokens

tensor([[25176,    62,    10,  6755,     9, 21356,   634,    10, 39546, 19650,
         31890,    25,    10,  1751,     4,    20,   313,  2288, 50264,   813,
             8,   916,   150,  5783,   418,     8,  7398,   257,  6058,     4,
           572,    10,   251,   803,     6,     5,   249,  2312,     7,  1349,
           123,   159,     8,   836,  1103,   136,   123,    13,    39, 50264,
          1713,     4, 50264,    22,   428,  4294, 19650, 50264, 17466, 13363,
         23907,     5,  1226,     8,   314,   171, 12144,    30,     5,  9818,
         18583,     9,    39,  3474,     4]])

In [217]:
span_lengths[cursor_idx]

2

In [219]:
span_lengths = span_lengths_es[0]
cursor_idx = 0
cursor = locations[1][cursor_idx]
cursor_offset = 0

comprehensive_hypotheses = []
while True:
    with torch.no_grad():
        comprehensive_hypotheses.append(torch.cat((test_sent_tokens[:, :cursor], test_sent_tokens[:, cursor+1:]), dim=-1))
        
        predictions = mlm(test_sent_tokens).logits
        predictions[:, :, [0,1,2,3,50264]] = -10000
        top3 = torch.topk(predictions[:, cursor,:], k=3, dim=-1).indices
        comprehensive_hypotheses.append(torch.cat((torch.tile(test_sent_tokens[:, :cursor], dims= (3,1)), top3.reshape(-1,1), torch.tile(test_sent_tokens[:, cursor+1:], dims= (3,1))), dim=-1))
        hypotheses = torch.cat((torch.tile(test_sent_tokens[:, :cursor], dims= (3,1)), top3.reshape(-1,1), torch.tile(test_sent_tokens[:, cursor:], dims= (3,1))), dim=-1)
        cursor_offset += 1
        
        iteration_count = max(span_lengths[cursor_idx], 5)
        
        for _ in range(iteration_count):
            hypotheses_texts = mlm_tokenizer.batch_decode(hypotheses)
            predictions = mlm(hypotheses).logits
            top3 = torch.topk(predictions[:, cursor + cursor_offset], k=3, dim=-1).indices
            print(torch.cat((torch.tile(hypotheses[:, :cursor + cursor_offset], dims= (3,1)), top3.reshape(-1,1), torch.tile(hypotheses[:, cursor + cursor_offset+1:], dims= (3,1))), dim=-1))
            hypotheses_scores = torch.softmax(grammar_model(torch.cat((torch.tile(hypotheses[:, :cursor + cursor_offset], dims= (3,1)), top3.reshape(-1,1), torch.tile(hypotheses[:, cursor + cursor_offset+1:], dims= (3,1))), dim=-1)).logits,dim=-1)
            # print(hypotheses_scores[: ,1])
            # break
            
            
            comprehensive_hypotheses.append(torch.cat((torch.tile(hypotheses[:, :cursor + cursor_offset], dims= (3,1)), top3.reshape(-1,1), torch.tile(hypotheses[:, cursor + cursor_offset+1:], dims= (3,1))), dim=-1))
            hypotheses = torch.cat((torch.tile(hypotheses[:, :cursor + cursor_offset], dims= (3,1)), top3.reshape(-1,1), torch.tile(hypotheses[:, cursor + cursor_offset:], dims= (3,1))), dim=-1)
            cursor_offset += 1
            if (hypotheses[:, cursor+cursor_offset-1] == hypotheses[:, cursor+cursor_offset]).sum() > 0:
                hypotheses = hypotheses[torch.randint(0, len(hypotheses), (10,)), :]
                break
            else:
                hypotheses = hypotheses[torch.randint(0, len(hypotheses), (10,)), :]
            
        
        
        break
        
        test_sent_tokens()
        test_sent_tokens

tensor([[25176,    62,    10,  6755,     9, 21356,   634,    10, 39546, 19650,
         31890,    25,    10,  1751,     4,    20,   313,  2288,  3711,     5,
           813,     8,   916,   150,  5783,   418,     8,  7398,   257,  6058,
             4,   572,    10,   251,   803,     6,     5,   249,  2312,     7,
          1349,   123,   159,     8,   836,  1103,   136,   123,    13,    39,
         50264,  1713,     4, 50264,    22,   428,  4294, 19650, 50264, 17466,
         13363, 23907,     5,  1226,     8,   314,   171, 12144,    30,     5,
          9818, 18583,     9,    39,  3474,     4],
        [25176,    62,    10,  6755,     9, 21356,   634,    10, 39546, 19650,
         31890,    25,    10,  1751,     4,    20,   313,  2288,  4487,   258,
           813,     8,   916,   150,  5783,   418,     8,  7398,   257,  6058,
             4,   572,    10,   251,   803,     6,     5,   249,  2312,     7,
          1349,   123,   159,     8,   836,  1103,   136,   123,    13,    39,


In [220]:
[mlm_tokenizer.batch_decode(x) for x in comprehensive_hypotheses]

[['holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly staff and customers while demanding money and valuables. After a long investigation, the police managed to track him down and bring charges against him for his<mask> activities.<mask> "broomstick<mask> saga captivated the nation and left many stunned by the audacity of his crimes.'],
 ['holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly threatened staff and customers while demanding money and valuables. After a long investigation, the police managed to track him down and bring charges against him for his<mask> activities.<mask> "broomstick<mask> saga captivated the nation and left many stunned by the audacity of his crimes.',
  'holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly attacked staff and customers while demanding money and valuables. After a long investigation, the police managed to track him dow

In [178]:
mlm_tokenizer.batch_decode(hypotheses_2)

['holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly threatened the<mask> staff and customers while demanding money and valuables. After a long investigation, the police managed to track him down and bring charges against him for his<mask> activities.<mask> "broomstick<mask> saga captivated the nation and left many stunned by the audacity of his crimes.',
 'holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly attacked both<mask> staff and customers while demanding money and valuables. After a long investigation, the police managed to track him down and bring charges against him for his<mask> activities.<mask> "broomstick<mask> saga captivated the nation and left many stunned by the audacity of his crimes.',
 'holding up a string of supermarkets using a broomstick disguised as a gun. The man reportedly assaulted store<mask> staff and customers while demanding money and valuables. After a long investigati

In [112]:
test_sent

"<mask><mask> know can actually hurt us.<mask><mask><mask> can lead to<mask><mask><mask> missed opportunities, and even dangerous situations. It's important to seek knowledge and be open to learning in order to avoid unnecessary harm or mistakes. Remember, sometimes the things we don't know are exactly what we need to discover to grow and make better choices in the future."

In [111]:
mlm_tokenizer.batch_decode(hypotheses_2)

["<s> alreadyWe<mask> know can actually hurt us.<mask> can lead to<mask> missed opportunities, and even dangerous situations. It's important to seek knowledge and be open to learning in order to avoid unnecessary harm or mistakes. Remember, sometimes the things we don't know are exactly what we need to discover to grow and make better choices in the future.</s>",
 "<s> doKnowing<mask> know can actually hurt us.<mask> can lead to<mask> missed opportunities, and even dangerous situations. It's important to seek knowledge and be open to learning in order to avoid unnecessary harm or mistakes. Remember, sometimes the things we don't know are exactly what we need to discover to grow and make better choices in the future.</s>",
 "<s> shouldNot<mask> know can actually hurt us.<mask> can lead to<mask> missed opportunities, and even dangerous situations. It's important to seek knowledge and be open to learning in order to avoid unnecessary harm or mistakes. Remember, sometimes the things we don

In [56]:
mask_info_dict

defaultdict(list,
            {0: [<re.Match object; span=(93, 99), match='<mask>'>,
              <re.Match object; span=(99, 105), match='<mask>'>],
             1: [<re.Match object; span=(265, 271), match='<mask>'>,
              <re.Match object; span=(271, 277), match='<mask>'>],
             2: [<re.Match object; span=(289, 295), match='<mask>'>],
             3: [<re.Match object; span=(307, 313), match='<mask>'>,
              <re.Match object; span=(313, 319), match='<mask>'>]})

In [44]:
mask.span()

(313, 319)

In [45]:
mask_info_dict

defaultdict(list,
            {0: [<re.Match object; span=(93, 99), match='<mask>'>,
              <re.Match object; span=(99, 105), match='<mask>'>,
              <re.Match object; span=(265, 271), match='<mask>'>,
              <re.Match object; span=(271, 277), match='<mask>'>,
              <re.Match object; span=(289, 295), match='<mask>'>,
              <re.Match object; span=(307, 313), match='<mask>'>,
              <re.Match object; span=(313, 319), match='<mask>'>]})